In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hash-code-dataset/100k.csv


In [20]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from scipy.sparse import *

In [21]:
#readintg the dataset
df = pd.read_csv("../input/hash-code-dataset/100k.csv")

In [22]:
df.head()

,id,result,hash
0,1,15.49,9f57575fd54cd441e962448537e155ace9555df10cd27f...
1,2,7.42,9f60c0b9439f8bdd27adb2053876d9fa36b7e3ca051a0d...
2,3,1.04,6e548074a57b39330de68ab00a2b42377a2d822fb1a777...
3,4,2.71,a840f5c02904a165d58da467afdfc9a7903cc54aef6f5e...
4,5,1.95,16fb2865a2c0fc828c7e789ee21262a02e88ea1c135b04...


In [23]:
#since the hashing yields an intermediate result we can see if that result is repeated anywhere and use that to train our data if so
print("The number of unique intermediary results is: ",df['result'].nunique())
print("The number of unique hashes is: ",df['hash'].nunique())
print("The number of actual elements in the df[result] column is: ",df['result'].count())

The number of unique intermediary results is:  5520
The number of unique hashes is:  100000
The number of actual elements in the df[result] column is:  100000


In [24]:
#finding the missing values in the dataset and sorting them in order
df.isnull().sum().sort_values(ascending = False)

id        0
result    0
hash      0
dtype: int64

In [25]:
#creating an additional column thatgets the length of each hash to see whether there is a correlation with the results column in the future
df['hash_length'] = df['hash'].str.len()
df.head()

,id,result,hash,hash_length
0,1,15.49,9f57575fd54cd441e962448537e155ace9555df10cd27f...,64
1,2,7.42,9f60c0b9439f8bdd27adb2053876d9fa36b7e3ca051a0d...,64
2,3,1.04,6e548074a57b39330de68ab00a2b42377a2d822fb1a777...,64
3,4,2.71,a840f5c02904a165d58da467afdfc9a7903cc54aef6f5e...,64
4,5,1.95,16fb2865a2c0fc828c7e789ee21262a02e88ea1c135b04...,64


In [26]:
#break down each of the hashed strings to see the components but for now let me just look at one
print(df['hash'][0])

9f57575fd54cd441e962448537e155ace9555df10cd27fa817bbcaf48a59ebcf


In [27]:
#from what we saw earlier we found out that there are a bunch of values that are duplicated in the results column
#selecting the duplicated rows
duplicate = df[df.duplicated('result')]

In [28]:
print(duplicate)

           id  result                                               hash  \
11         12    1.10  0f123b0ca3edde4a1ed71dc52220131779782c6bdceb41...   
13         14    1.04  d6f7738f38f81fcdc8db9fb55ddb5b243372299603d7e1...   
19         20    1.66  4a474838d2b2d837474e1cc028997ff38aa28fd03c24da...   
29         30    2.81  5979dcae5d4aa259f4519e1351703e8d9479cc32dcd092...   
32         33    2.04  5ee98b664ab77b2f1dcdc576bef312c740a701e00518eb...   
...       ...     ...                                                ...   
99994   99995   10.22  35ef64d8afe7dbc3be05c817d3321b92a6fa54dd567f48...   
99995   99996    1.96  ca5bf536a33b45e871b34a6a1ff1a01ecb5251bf12de4a...   
99996   99997    2.40  1da510bfbd148cb59a425467c87383c6f309afdad167a0...   
99998   99999    1.01  3543f643a4319213e7fb48105c9ebad08859ea318badc9...   
99999  100000    6.15  f8dd0c706ca4fc5bf48d2f8e62959512f5308d863ea2a8...   

       hash_length  
11              64  
13              64  
19              64  
29 

In [29]:
#from what we have in the duplicated we get a concept of what is called collission which is 
#multiple inputs can hash to the same output as is the case here
"""the game result that we have as our value for the result column is only related to the sha256 crypto
by the number derivation function which is done over and over again, so whatever we will find when we ran through the 
model is most likely the number derivation function relationship between the hash 256 and the result

You are supposed to find the relationsgip between the strings for you are supposed to generate a 64 bit string
that has not been seeing

sth that I learnt yesterday is that hasing functions are always deteerministic which means that the same input 
should always yield the same output even when done over and over again

To decipher this we have to start with what is a sha256 algos it is typically a 32 byte signature expressed as a 
64 characters strings"""

'the game result that we have as our value for the result column is only related to the sha256 crypto\nby the number derivation function which is done over and over again, so whatever we will find when we ran through the \nmodel is most likely the number derivation function relationship between the hash 256 and the result\n\nYou are supposed to find the relationsgip between the strings for you are supposed to generate a 64 bit string\nthat has not been seeing\n\nsth that I learnt yesterday is that hasing functions are always deteerministic which means that the same input \nshould always yield the same output even when done over and over again\n\nTo decipher this we have to start with what is a sha256 algos it is typically a 32 byte signature expressed as a \n64 characters strings'

In [30]:
# I want to sample a few hashes and see what they contain ehn they are broken down
print("This is the first hashed output: ",df['hash'][0])
print("This is the hundredth hashed output: ",df['hash'][100])
print("This is the second hundredth hashed output: ",df['hash'][200])
print("This is the third hundredth hashed output: ",df['hash'][300])
print("This is the 77th hashed output: ",df['hash'][77])

This is the first hashed output:  9f57575fd54cd441e962448537e155ace9555df10cd27fa817bbcaf48a59ebcf
This is the hundredth hashed output:  5ff21f42cf5f871ad3b0918a5a6afb51e676f4e428aaedede38d7371ad40dd94
This is the second hundredth hashed output:  a8cd32e233ee4fe72fcc765c3a69ed87a33225664eb4798b22105e32d44fac29
This is the third hundredth hashed output:  f88e7616df0d888a5a2ecce7dd99978c6c9cb89633d290c6c1cd1ac57d0fc683
This is the 77th hashed output:  f6aff0a84bdf3734485681f909c51be43eb0ef5b110d81aec873b832da244b85


In [31]:
#count columns for the hexadecimals
zeros_count = []
ones_count = []
two_count = []
three_count = []
four_count = []
five_count = []
six_count = []
seven_count = []
eight_count = []
nine_count = []
a_count = []
b_count = []
c_count = []
d_count = []
e_count = []
f_count = []
#function for the count
for i in df['hash']:
    zeros_counter = i.count('0')
    zeros_count.append(zeros_counter)
    ones_counter = i.count('1')
    ones_count.append(ones_counter)
    two_counter = i.count('2')
    two_count.append(two_counter)
    three_counter = i.count('3')
    three_count.append(three_counter)
    four_counter = i.count('4')
    four_count.append(four_counter)
    five_counter = i.count('5')
    five_count.append(five_counter)
    six_counter = i.count('6')
    six_count.append(six_counter)
    seven_counter = i.count('7')
    seven_count.append(seven_counter)
    eight_counter = i.count('8')
    eight_count.append(eight_counter)
    nine_counter = i.count('9')
    nine_count.append(nine_counter)
    a_counter = i.count('a')
    a_count.append(a_counter)
    b_counter = i.count('b')
    b_count.append(b_counter)
    c_counter = i.count('c')
    c_count.append(c_counter)
    d_counter = i.count('d')
    d_count.append(d_counter)
    e_counter = i.count('e')
    e_count.append(e_counter)
    f_counter = i.count('f')
    f_count.append(f_counter)

print(len(five_count))

100000


In [33]:
df['zeros_count'] = pd.DataFrame (zeros_count)

In [ ]:
df['ones_count'] = pd.DataFrame (ones_count)
df['two_count'] = pd.DataFrame (two_count)
df['three_count'] = pd.DataFrame (three_count)
df['four_count'] = pd.DataFrame (four_count)
df['five_count'] = pd.DataFrame (five_count)
df['six_count'] = pd.DataFrame (six_count)
df['seven_count'] = pd.DataFrame (seven_count)
df['eight_count'] = pd.DataFrame (eight_count)
df['nine_count'] = pd.DataFrame (nine_count)
df['a_count'] = pd.DataFrame (a_count)
df['b_count'] = pd.DataFrame (b_count)
df['c_count'] = pd.DataFrame (c_count)
df['d_count'] = pd.DataFrame (d_count)
df['e_count'] = pd.DataFrame (e_count)
df['f_count'] = pd.DataFrame (f_count)

In [18]:
df.head()

,f_count
0,6
1,4
2,2
3,7
4,2
